In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
print(es_client.info())

{'name': '1f08c3fad565', 'cluster_name': 'docker-cluster', 'cluster_uuid': '3SEmyz2SRRO-Z0vJO3tigA', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [8]:
!curl localhost:9200

{
  "name" : "1f08c3fad565",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "3SEmyz2SRRO-Z0vJO3tigA",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Q2 Indexing the data

In [6]:
from elasticsearch import Elasticsearch, NotFoundError

es_client = Elasticsearch("http://localhost:9200")

index_name = "course-questions"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

# Delete if exists
try:
    es_client.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")
except NotFoundError:
    print("Index not found, skipping delete.")

# Create the index
es_client.indices.create(
    index=index_name,
    settings=index_settings["settings"],
    mappings=index_settings["mappings"]
)
print(" Index created.")


Deleted index: course-questions
 Index created.


Answer 2: Index

Q3 Searching

In [14]:
query = "How do I execute a command in a running docker container?"

def elastic_search(query):
    
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    # Searhing using the query
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append({
            "_source": hit['_source'],
            "_score": hit['_score']
        })

    return result_docs

elastic_search(query)

[{'_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  '_score': 84.050095},
 {'_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\

Answer 3: 84.05

Q4 Filtering

In [15]:
def elastic_search_filtered(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    # Searhing using the query
    response = es_client.search(index=index_name,body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

elastic_search_filtered(query)


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

Answer 4 :How do I copy files from a different folder into docker container’s working directory?

Q5 Building a prompt

In [16]:
def build_prompt(query, search_results):
    context_entries = []
    context_template = """
Q: {question}
A: {text}
""".strip()

    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    for record in search_results:
        # print("Printing a record: ", record)
        formatted_entry = context_template.format(question=record['question'], text=record['text'])
        #print("Printing the formatted entry: ", formatted_entry)
        context_entries.append(formatted_entry)

    context = "\n\n".join(context_entries)

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

final_prompt = build_prompt(query, elastic_search_filtered(query))

print(final_prompt)
len(final_prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

1462

Answer 5: 1462

Q6 Tokens

In [17]:
import sys
sys.path.append('/home/dmitry/Documents/Programming/Python/llm-zoomcamp/.venv/lib/python3.12/site-packages/')
import tiktoken
# Using tokenization algorithm for gpt-4o: https://community.openai.com/t/whats-the-new-tokenization-algorithm-for-gpt-4o/746708
# encoding = tiktoken.get_encoding("o200k_base")
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(final_prompt)
len(tokens)

322

Answer 6: 322

In [33]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="openai/gpt-4o",
    messages=[
        {"role": "user", "content": final_prompt}
    ],
    temperature=0.3,
    max_tokens=300  # limit output tokens so it's affordable
)

answer = response.choices[0].message.content
print("AI Answer:\n", answer)


AI Answer:
 To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID by using `docker ps`. Then, execute the command in the specific container using the following syntax:

```bash
docker exec -it <container-id> <command>
```

For example, to start a bash session in the container, you would use:

```bash
docker exec -it <container-id> bash
```


In [30]:
# Define values
input_tokens_per_request = 150
output_tokens_per_request = 250
num_requests = 1000

# Prices (as of June 17)
input_price_per_1k = 0.005   # $ per 1000 input tokens
output_price_per_1k = 0.015  # $ per 1000 output tokens

# Total tokens
total_input_tokens = input_tokens_per_request * num_requests
total_output_tokens = output_tokens_per_request * num_requests

# Total cost
input_cost = (total_input_tokens / 1000) * input_price_per_1k
output_cost = (total_output_tokens / 1000) * output_price_per_1k
total_cost = input_cost + output_cost

# Print breakdown
print(f"Total input tokens: {total_input_tokens} → ${input_cost:.2f}")
print(f"Total output tokens: {total_output_tokens} → ${output_cost:.2f}")
print(f" Total estimated cost for {num_requests} requests: ${total_cost:.2f}")


Total input tokens: 150000 → $0.75
Total output tokens: 250000 → $3.75
 Total estimated cost for 1000 requests: $4.50


In [35]:
encoding = tiktoken.encoding_for_model("gpt-4")  # GPT-4o uses same encoding as GPT-4 for now

input_tokens = encoding.encode(final_prompt)
output_tokens = encoding.encode(answer)

num_input_tokens = len(input_tokens)
num_output_tokens = len(output_tokens)

print("\n Token usage:")
print("Input tokens per request:", num_input_tokens)
print("Output tokens per request:", num_output_tokens)

#  Cost estimation
price_per_1k_input = 0.005
price_per_1k_output = 0.015
num_requests = 1000

total_input_tokens = num_input_tokens * num_requests
total_output_tokens = num_output_tokens * num_requests

input_cost = (total_input_tokens / 1000) * price_per_1k_input
output_cost = (total_output_tokens / 1000) * price_per_1k_output
total_cost = input_cost + output_cost

print("\n Estimated cost for 1000 requests:")
print("Total input tokens:", total_input_tokens)
print("Total output tokens:", total_output_tokens)
print("Input cost: $", round(input_cost, 2))
print("Output cost: $", round(output_cost, 2))
print("Total estimated cost: $", round(total_cost, 2))


 Token usage:
Input tokens per request: 325
Output tokens per request: 92

 Estimated cost for 1000 requests:
Total input tokens: 325000
Total output tokens: 92000
Input cost: $ 1.62
Output cost: $ 1.38
Total estimated cost: $ 3.0
